In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter("ignore")

In [2]:
text = pd.read_csv(r"C:\Users\Samson\Desktop\twitter_training.csv")
text.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


# Data Cleaning

In [3]:
text.shape

(74681, 4)

In [4]:
text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   2401                                                   74681 non-null  int64 
 1   Borderlands                                            74681 non-null  object
 2   Positive                                               74681 non-null  object
 3   im getting on borderlands and i will murder you all ,  73995 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [5]:
text['Positive'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [6]:
text['Positive'].value_counts()

Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: Positive, dtype: int64

In [7]:
text.describe()

,2401
count,74681.000000
mean,6432.640149
std,3740.423819
min,1.000000
25%,3195.000000
50%,6422.000000
75%,9601.000000
max,13200.000000


In [8]:
text.isnull().sum()

2401                                                       0
Borderlands                                                0
Positive                                                   0
im getting on borderlands and i will murder you all ,    686
dtype: int64

In [9]:
text.duplicated().sum()
text.drop_duplicates(inplace=True)

In [10]:
text.drop(columns=['2401','Borderlands'],inplace=True,axis=1)

In [11]:
text.rename(columns = {'Positive':'Sentiment',"im getting on borderlands and i will murder you all ," : "text"}, inplace=True)

In [12]:
text.head()

,Sentiment,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [13]:
text['Sentiment'].value_counts()

Negative      21787
Positive      19810
Neutral       17800
Irrelevant    12584
Name: Sentiment, dtype: int64

In [14]:
text.isnull().sum()

Sentiment      0
text         326
dtype: int64

In [15]:
text.dropna(inplace=True)

In [16]:
text.isnull().sum()

Sentiment    0
text         0
dtype: int64

# Vectorization

In [17]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [18]:
import re

def clean_text(df):
    text['text'] = df['text'].apply(lambda text: re.sub(r'[^\w\s]', '', text))
    text['text'] = df['text'].apply(lambda text: re.sub(r'http\S+', '', text))
    text['text'] = df['text'].apply(lambda text: re.sub(r'\s+', ' ', text).strip())

# Assuming 'text' is a column in your DataFrame
clean_text(text)


In [19]:
print(text)

      Sentiment                                               text
0      Positive  I am coming to the borders and I will kill you...
1      Positive  im getting on borderlands and i will kill you all
2      Positive  im coming on borderlands and i will murder you...
3      Positive  im getting on borderlands 2 and i will murder ...
4      Positive  im getting into borderlands and i can murder y...
...         ...                                                ...
74676  Positive  Just realized that the Windows partition of my...
74677  Positive  Just realized that my Mac window partition is ...
74678  Positive  Just realized the windows partition of my Mac ...
74679  Positive  Just realized between the windows partition of...
74680  Positive  Just like the windows partition of my Mac is l...

[71655 rows x 2 columns]


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
tf = TfidfVectorizer(max_features=10000,stop_words='english')
X = tf.fit_transform(text['text']).toarray()
lbl = LabelEncoder()
y = lbl.fit_transform(text['Sentiment'])

In [22]:
# Converting text data into array
x_arr = np.array(X)
y_arr = np.array(y)

# Train Test Split

In [23]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,
                                                random_state=0)

# Modeling

# Naive Bayes

In [24]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train,y_train)

MultinomialNB()

In [25]:
ypred_test = model.predict(X_test)
ypred_train = model.predict(X_train)

In [26]:
from sklearn.metrics import accuracy_score
print("Train Accuracy:",accuracy_score(y_train,ypred_train))
print("Test Accutacy:",accuracy_score(y_test,ypred_test))

Train Accuracy: 0.7309503872723466
Test Accutacy: 0.6813899937199079


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnf = RandomForestClassifier(criterion='entropy')
rnf.fit(X_train,y_train)

In [ ]:
ypred_test = rnf.predict(X_test)
ypred_train = rnf.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score
print("Train Accuracy:",accuracy_score(y_train,ypred_train))
print("Test Accutacy:",accuracy_score(y_test,ypred_test))

# SVM

In [ ]:
from sklearn.svm import SVC
sv = SVC()
sv.fit(X_train,y_train)

In [ ]:
ypred_test = sv.predict(X_test)
ypred_train = sv.predict(X_train)

In [ ]:
from sklearn.metrics import accuracy_score
print("Train Accuracy:",accuracy_score(y_train,ypred_train))
print("Test Accutacy:",accuracy_score(y_test,ypred_test))